# Tabu Search using short term memory
# Tabu Search Basic Algorithm
1. TS was first proposed by Glover in 1986 and was also developed by Hansen in parallel, since then TS has been successfully applied to many optimization problems
2. TS tries to find the best admissible solution in the neighborhood of the current solution in each iteration, considering recent solutions as ‘Tabu’ to prevent cycling.

Step 0: The initial step is to create an initial solution

Step 1: Create the list of candidate solutions from the initial solution - the neighborhood

Step 2: From the neighborhood solutions list created in step 1, choose the best admissible (Non-tabu or meets aspiration criteria) solution by checking each solution as in the diagram below:
<img src="tabu.png">

*Tabu list* 
1. TS uses to record the recent solutions and prevents them to reoccur for a specified number of iterations
2. *Tabu Tenure*: This defines the size of the Tabu list, i.e., for how many iterations a solution component is kept as Tabu
3. Tabu Attributes defines the solution component kept in the tabu list.

*Aspiration Criteria*
1. If the solution found in the current iteration has a value (objective function value) better than the currently-known best solution’s value, but the move (like don’t swap job 3 with 1) is Tabu, we can use the Aspiration Criteria to override the tabu state of this move

Step 3: Check the defined stopping criteria, this can be the max number of iterations reached or the time of running, if the stopping criteria are not met, go to step 4, if the stopping criteria are met terminate and return the best solution.

Step 4: Update Tabu list, Aspiration Criteria, and go to Step 1

# Read excel file/data
1. install openpyxl using pip

In [35]:
import pandas as pd
import random as rd
from itertools import combinations
import math

def input_data(Path):
    '''Takes the path of the excel file of the SMTWTP instances.
    Returns a dict of jobs number as Key and weight, processing time (hours) and due date (hours) as values.
    '''
    return pd.read_excel(Path, engine='openpyxl', names=['Job', 'weight', "processing_time", "due_date"], index_col=0).to_dict('index')

instance_dict = input_data('Instance_10.xlsx')
instance_dict


{1: {'weight': 0.5, 'processing_time': 1.5, 'due_date': 4},
 2: {'weight': 0.8, 'processing_time': 2.2, 'due_date': 4},
 3: {'weight': 0.6, 'processing_time': 0.5, 'due_date': 4},
 4: {'weight': 0.4, 'processing_time': 1.6, 'due_date': 6},
 5: {'weight': 0.4, 'processing_time': 3.5, 'due_date': 6},
 6: {'weight': 0.1, 'processing_time': 2.1, 'due_date': 6},
 7: {'weight': 0.2, 'processing_time': 2.5, 'due_date': 8},
 8: {'weight': 0.5, 'processing_time': 0.6, 'due_date': 8},
 9: {'weight': 0.3, 'processing_time': 3.2, 'due_date': 8},
 10: {'weight': 0.6, 'processing_time': 5.2, 'due_date': 8}}

# objective function

Objective function: Task scheduling. 

Problem: The single machine total weighted tardiness problem (delay minimization)

Job	weight	processing time (hours)	due date (hours)
1	0.5	    1.5	                    4
2	0.8	    2.2	                    4
3	0.6		0.5		                4
4	0.4	    1.6		                6
5	0.4	    3.5		                6
6	0.1	    2.1		                6
7	0.2	    2.5		                8
8	0.5	    0.6		                8
9	0.3	    3.2		                8
10	0.6	    5.2		                8

Example order of job: [1,2,5,6,8,9,10,3,4,7]

We have a single machine that can handle only one job at a time, and there is an N number of jobs (or tasks) to be processed without interruption on the machine.

Each job i ∈ N requires an integer processing time Pi, and has a positive weight Wi indicates the importance of the job and a due date di. If we assume that the machine becomes available for processing at time zero, we can indicate the completion time of job i as Ci and the tardiness of the job can be calculated as Ti = max{Ci − di, 0}, so if the job is processed before its due date Ci ≤ di, the will be no tardiness T = 0.

The objective is to order the N jobs in a way that minimizes the total weighted tardiness of the whole process, i.e., min ∑WiTi, notice that if a job has a higher weight the penalty of tardiness will be higher.

Suppose that the current solution is solution_0 = [1,2,3,4,5,6,7,8,9,10]

E.g. for Job 1, assume t=0, where we just start, i=0 
C_i = t + dict[job]["processing_time"] = 1.5
d_i = dict[job]["due_date"] = 4
T_i = max(0, C_i - d_i) = max(0, 1.5-4) = 0
W_i = dict[job]["weight"] = 0.5

objfun_value +=  W_i * T_i = 0.5 * 0 = 0.0

t= C_i = 1.5

next, for job 2
C_i = t + dict[job]["processing_time"] = 1.5 + 2.2 = 3.7
d_i = dict[job]["due_date"] = 4
T_i = max(0, C_i - d_i) = max(0, 3.7-4) = 0
W_i = dict[job]["weight"] = 0.8

objfun_value +=  W_i * T_i = 0.8 * 0 = 0.0

t= C_i = 3.7

next, for job 3, 
C_i = t + dict[job]["processing_time"] = 3.7 + 0.5 = 4.2
d_i = dict[job]["due_date"] = 4
T_i = max(0, C_i - d_i) = max(0, 4.2-4) = 0.2
W_i = dict[job]["weight"] = 0.6

objfun_value +=  W_i * T_i = 0.6 * 0.2 = 0.12

t= C_i = 4.2

and so on until job 10, then then sum, ∑WiTi (objective function), is: 
∑WiTi = 18.259999999999998 

In [36]:
def Objfun(instance_dict, solution, show = False):
    '''Takes a set of scheduled jobs, dict (input data)
    Return the objective function value of the solution
    '''
    dict = instance_dict
    t = 0   #starting time
    objfun_value = 0
    for job in solution:
        C_i = t + dict[job]["processing_time"]  # Completion time
        d_i = dict[job]["due_date"]   # due date of the job
        T_i = max(0, C_i - d_i)    #tardiness for the job
        W_i = dict[job]["weight"]  # job's weight

        objfun_value +=  W_i * T_i
        print(objfun_value)
        t = C_i
    if show == True:
        print("#"*4, "The Objective function value for {} \nsolution schedule is: {}".format(solution ,objfun_value),"#"*4,"\n")
    return objfun_value

In [37]:
solution_0 = [1,2,3,4,5,6,7,8,9,10]
solution_1 = [1,2,5,6,8,9,10,3,4,7]
solution_2 = [2,3,5,10,6,8,9,4,7,1]

Objfun(instance_dict, solution_0, show=True)
Objfun(instance_dict, solution_1, show=True)
Objfun(instance_dict,solution_2, show=True);

0.0
0.0
0.1200000000000001
0.1200000000000001
1.4400000000000004
1.9800000000000004
3.1600000000000006
6.41
9.32
18.259999999999998
#### The Objective function value for [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
solution schedule is: 18.259999999999998 #### 

0.0
0.0
0.4800000000000001
0.8100000000000002
1.7600000000000002
3.290000000000001
9.470000000000002
18.35
24.110000000000003
27.090000000000003
#### The Objective function value for [1, 2, 5, 6, 8, 9, 10, 3, 4, 7] 
solution schedule is: 27.090000000000003 #### 

0.0
0.0
0.08000000000000007
2.12
2.87
5.92
8.71
13.870000000000001
16.55
26.0
#### The Objective function value for [2, 3, 5, 10, 6, 8, 9, 4, 7, 1] 
solution schedule is: 26.0 #### 



# Function to generate initial random solution

In [ ]:
def get_InitialSolution(instance_dict, seed, show=False):
    n_jobs = len(instance_dict) # Number of jobs
    # Producing a random schedule of jobs
    initial_solution = list(range(1, n_jobs+1))
    rd.seed(seed)
    rd.shuffle(initial_solution)
    if show == True:
        print("initial Random Solution: {}".format(initial_solution))
    return initial_solution

# Function to make adjustment to solution (e.g swap).
*could also be done using mutation where some random probability can be generated and if the probability > 0.5, then do mutation (change the value of certain job in the list to other value)

In [ ]:
def SwapMove(solution, i ,j):
    '''Takes a list (solution)
    returns a new neighbor solution with i, j swapped
    '''
    solution = solution.copy()
    # job index in the solution:
    i_index = solution.index(i)
    j_index = solution.index(j)
    #Swap
    solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
    return solution

initial_solution = get_InitialSolution(instance_dict, 2021, show=True);
print('Neighbor Solution: {}'.format(SwapMove(initial_solution, 10, 5)))

# Full implementation of Tabu Search using class

In [38]:
import pandas as pd
import random as rd
from itertools import combinations
import math

class TS():
    def __init__(self, Path, seed, tabu_tenure):
        self.Path = Path
        self.seed = seed
        self.tabu_tenure = tabu_tenure
        self.instance_dict = self.input_data()
        self.Initial_solution = self.get_InitialSolution()
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()


    def input_data(self):
        '''Takes the path of the excel file of the SMTWTP instances.
        Returns a dict of jobs number as Key and weight, processing time (hours) and due date (hours) as values.
        '''
        return pd.read_excel(self.Path, engine='openpyxl', names=['Job', 'weight', "processing_time", "due_date"],
                                 index_col=0).to_dict('index')

    def get_tabuestructure(self):
        '''Takes a dict (input data)
        Returns a dict of tabu attributes(pair of jobs that are swapped) as keys and [tabu_time, MoveValue]
        '''
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict

    def get_InitialSolution(self, show=False):
        n_jobs = len(self.instance_dict) # Number of jobs
        # Producing a random schedule of jobs
        initial_solution = list(range(1, n_jobs+1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        if show == True:
            print("initial Random Solution: {}".format(initial_solution))
        return initial_solution

    def Objfun(self, solution, show = False):
        '''Takes a set of scheduled jobs, dict (input data)
        Return the objective function value of the solution
        '''
        dict = self.instance_dict
        t = 0   #starting time
        objfun_value = 0
        for job in solution:
            C_i = t + dict[job]["processing_time"]  # Completion time
            d_i = dict[job]["due_date"]   # due date of the job
            T_i = max(0, C_i - d_i)    #tardiness for the job
            W_i = dict[job]["weight"]  # job's weight

            objfun_value +=  W_i * T_i
            t = C_i
        if show == True:
            print("\n","#"*8, "The Objective function value for {} solution schedule is: {}".format(solution ,objfun_value),"#"*8)
        return objfun_value

    def SwapMove(self, solution, i ,j):
        '''Takes a list (solution)
        returns a new neighbor solution with i, j swapped
       '''
        solution = solution.copy()
        # job index in the solution:
        i_index = solution.index(i)
        j_index = solution.index(j)
        #Swap
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution

    def TSearch(self):
        '''The implementation Tabu search algorithm with 
        short-term memory and pair_swap as Tabu attribute.
        '''
        # Parameters:
        tenure =self.tabu_tenure
        tabu_structure = self.get_tabuestructure()  # Initialize the data structures
        best_solution = self.Initial_solution
        best_objvalue = self.Objfun(best_solution)
        current_solution = self.Initial_solution
        current_objvalue = self.Objfun(current_solution)

        print("#"*30, "Short-term memory TS with Tabu Tenure: {}\nInitial Solution: {}, Initial Objvalue: {}".format(
            tenure, current_solution, current_objvalue), "#"*30, sep='\n\n')
        iter = 1
        Terminate = 0
        while Terminate < 100:
            print('\n\n### iter {}###  Current_Objvalue: {}, Best_Objvalue: {}'.format(iter, current_objvalue,
                                                                                    best_objvalue))
            # Searching the whole neighborhood of the current solution:
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue

            # Admissible move
            while True:
                # select the move with the lowest ObjValue in the neighborhood (minimization)
                best_move = min(tabu_structure, key =lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]["MoveValue"]
                tabu_time = tabu_structure[best_move]["tabu_time"]
                
                # Not Tabu
                if tabu_time < iter:
                    # make the move
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    # Best Improving move
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Best Improving => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                    else:
                        print("   ##Termination: {}## best_move: {}, Objvalue: {} => Least non-improving => "
                              "Admissible".format(Terminate,best_move,
                                                                                                           current_objvalue))
                        Terminate += 1
                    # update tabu_time for the move
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                
                # If tabu
                else:
                    # Aspiration
                    if MoveValue < best_objvalue:
                        # make the move
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Aspiration => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]["MoveValue"] = float('inf')
                        print("   best_move: {}, Objvalue: {} => Tabu => Inadmissible".format(best_move,
                                                                                              current_objvalue))
                        continue
        print('#'*50 , "Performed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,best_objvalue), sep="\n")
        return tabu_structure, best_solution, best_objvalue


test = TS(Path="Instance_10.xlsx", seed = 2012, tabu_tenure=3)

##############################

Short-term memory TS with Tabu Tenure: 3
Initial Solution: [4, 7, 9, 1, 5, 3, 10, 6, 8, 2], Initial Objvalue: 39.080000000000005

##############################


### iter 1###  Current_Objvalue: 39.080000000000005, Best_Objvalue: 39.080000000000005
   best_move: (2, 7), Objvalue: 26.13 => Best Improving => Admissible


### iter 2###  Current_Objvalue: 26.13, Best_Objvalue: 26.13
   best_move: (8, 9), Objvalue: 17.930000000000003 => Best Improving => Admissible


### iter 3###  Current_Objvalue: 17.930000000000003, Best_Objvalue: 17.930000000000003
   best_move: (3, 4), Objvalue: 14.960000000000003 => Best Improving => Admissible


### iter 4###  Current_Objvalue: 14.960000000000003, Best_Objvalue: 14.960000000000003
   best_move: (4, 5), Objvalue: 14.200000000000003 => Best Improving => Admissible


### iter 5###  Current_Objvalue: 14.200000000000003, Best_Objvalue: 14.200000000000003
   best_move: (6, 7), Objvalue: 13.830000000000002 => Best Improving 

Reference
https://medium.com/swlh/tabu-search-in-python-3199c44d44f1